In [5]:
import os
import shutil
import yaml
from sklearn.model_selection import train_test_split
import glob 
from ultralytics import YOLO
import torch_directml

In [6]:
try:
    device = torch_directml.device()
    print(f"DirectML Device detected: {device}")
except ImportError:
    pass

DirectML Device detected: privateuseone:0


In [7]:
SOURCE_DIR = os.path.join(os.getcwd(), 'manual_annotation_workspace')
DEST_DIR = os.path.join(os.getcwd(), 'fdm_manual_dataset')

def organize_dataset():
    
    images = []
    # Scan for common image extensions
    for ext in ['*.jpg', '*.png', '*.jpeg']:
        # FIXED: Using glob.glob() to avoid the TypeError
        images.extend(glob.glob(os.path.join(SOURCE_DIR, ext)))
    
    annotated_pairs = []
    
    print(f">>> Scanning {SOURCE_DIR} for annotated images...")
    
    for img_path in images:
        base_name = os.path.splitext(img_path)[0]
        txt_path = base_name + ".txt"
        
        # We only keep images that have a matching .txt label file
        if os.path.exists(txt_path):
            annotated_pairs.append((img_path, txt_path))
        else:

            print(f"Skipping un-annotated image: {os.path.basename(img_path)}")

    if not annotated_pairs:
        raise ValueError("No annotated (.txt) files found! Did you save them in YOLO format inside the 'manual_annotation_workspace' folder?")

    
    classes_file = os.path.join(SOURCE_DIR, 'classes.txt')
    if not os.path.exists(classes_file):
        raise FileNotFoundError("classes.txt not found. LabelImg creates this automatically when you save. Did you define classes?")
    
    with open(classes_file, 'r') as f:
        class_names = [line.strip() for line in f.readlines() if line.strip()]

    print(f"Found {len(annotated_pairs)} annotated images.")
    print(f"Classes found: {class_names}")

    
    train_pairs, val_pairs = train_test_split(annotated_pairs, test_size=0.1, random_state=42)

    
    if os.path.exists(DEST_DIR):
        try:
            shutil.rmtree(DEST_DIR)
        except:
            pass 
            
    for split, pairs in [('train', train_pairs), ('val', val_pairs)]:
        img_dest = os.path.join(DEST_DIR, 'images', split)
        lbl_dest = os.path.join(DEST_DIR, 'labels', split)
        os.makedirs(img_dest, exist_ok=True)
        os.makedirs(lbl_dest, exist_ok=True)
        
        for img_src, txt_src in pairs:
            shutil.copy(img_src, os.path.join(img_dest, os.path.basename(img_src)))
            shutil.copy(txt_src, os.path.join(lbl_dest, os.path.basename(txt_src)))

    # 5. Create data.yaml
    yaml_content = {
        'path': DEST_DIR,
        'train': 'images/train',
        'val': 'images/val',
        'nc': len(class_names),
        'names': {i: name for i, name in enumerate(class_names)}
    }
    
    yaml_path = os.path.join(DEST_DIR, 'data.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f)

    print(f"\n>>> SUCCESS! Dataset ready at: {DEST_DIR}")
    return yaml_path


organize_dataset()

>>> Scanning c:\Users\Florian Caspar\Desktop\Desly\DefectClassification-main\manual_annotation_workspace for annotated images...
Found 198 annotated images.
Classes found: ['cracking', 'layer_shifting', 'off_platform', 'stringing']

>>> SUCCESS! Dataset ready at: c:\Users\Florian Caspar\Desktop\Desly\DefectClassification-main\fdm_manual_dataset


'c:\\Users\\Florian Caspar\\Desktop\\Desly\\DefectClassification-main\\fdm_manual_dataset\\data.yaml'

In [8]:
DATASET_YAML = os.path.join(os.getcwd(), 'fdm_manual_dataset', 'data.yaml')
#MODELS = ['yolov5m6u.pt', 'yolov8m.pt', 'yolov8x.pt', 'yolov9e.pt' ]
MODELS = ['yolov8m.pt', 'yolov8x.pt', 'yolov9e.pt' ]
dml_device = torch_directml.device()
print(f"Using device: {dml_device}")

HYPERPARAMS = {
    'epochs': 100,      # Set to 5 or 10 if you just want a quick test run!
    'batch': 16,        # Paper used 64, but 16 is safer for typical GPUs
    'imgsz': 640,
    'optimizer': 'Adam', 
    'lr0': 0.0001,
    'lrf': 0.2,
    'momentum': 0.9,
    'weight_decay': 0.0005,
    'augment': True      
}

def train_and_validate(model, project_name):
    model_name = model.removesuffix(".pt")
    model = YOLO(model) 

    print(f">>> Starting training using: {DATASET_YAML}")

    # 2. Train
    # Ultralytics automatically saves the best model during training
    results = model.train(
        data=DATASET_YAML,
        # device=dml_device,
        # amp=False,
        project=project_name,
        name=model_name,
        exist_ok=True,        # Overwrite if exists
        verbose=True,
        **HYPERPARAMS
    )

    # 3. Validate
    print("\n>>> Running Validation...")
    metrics = model.val()

    # 4. Print Accuracy Scores
    # In Object Detection, "Accuracy" is measured by mAP (Mean Average Precision)
    print("\n" + "="*30)
    print("      FINAL RESULTS")
    print("="*30)
    
    # mAP@0.5: How accurate the boxes are (Standard Metric)
    map50 = metrics.results_dict['metrics/mAP50(B)']
    # mAP@0.5-0.95: Strict accuracy (Requires very tight boxes)
    map50_95 = metrics.results_dict['metrics/mAP50-95(B)']
    precision = metrics.results_dict['metrics/precision(B)']
    recall = metrics.results_dict['metrics/recall(B)']

    print(f"Precision: {precision:.4f} (Few false alarms?)")
    print(f"Recall:    {recall:.4f} (Found all defects?)")
    print(f"mAP@0.5:   {map50:.4f} (Overall Accuracy Score)")
    print(f"mAP@0.95:  {map50_95:.4f} (Strict Accuracy)")
    
    # 5. Locate Saved Model
    # It is saved automatically, but we print the exact path for you
    best_weight_path = os.path.join(os.getcwd(), project_name, model_name , 'weights', 'best.pt')
    
    print("\n" + "="*30)
    print(f"MODEL SAVED AT: {best_weight_path}")
    print("="*30)

    return best_weight_path

for model in MODELS:
    print(f"Training model : {model}")
    project_name = f"{'fdm_manual_training'}"
    saved_path = train_and_validate(model, project_name)

Using device: privateuseone:0
Training model : yolov8m.pt
>>> Starting training using: c:\Users\Florian Caspar\Desktop\Desly\DefectClassification-main\fdm_manual_dataset\data.yaml
Ultralytics 8.3.245  Python-3.11.1 torch-2.4.1+cpu CPU (AMD RYZEN AI MAX+ 395 w/ Radeon 8060S)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\Users\Florian Caspar\Desktop\Desly\DefectClassification-main\fdm_manual_dataset\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.2, mas

KeyboardInterrupt: 